# Проект команды 306 - Проектный семинар - №4 (Генерация классов)

Состав команды:

1. Алиев Хайрутдин Аллилович
2. Зубов Дмитрий Сергеевич
3. Курбанов Иван Сергеевич
4. Лухнев Игорь Дмитриевич
5. Шишков Максим Алексеевич

## TL;DR
---
В данном документе мы проводим применение различных предобученных нейронных сетей для извлечения признаков из картинок.

---

## Предварительные зависимости
В следующей ячейке будут установлены необходимые библиотеки через `pip`

In [1]:
# Use !pip install -q [your-package]
!pip install -q PyGithub
!pip install -q fsspec
!pip install -q catboost

     |████████████████████████████████| 291 kB 5.2 MB/s 
     |████████████████████████████████| 856 kB 40.1 MB/s 
     |████████████████████████████████| 134 kB 4.7 MB/s 
     |████████████████████████████████| 76.1 MB 51 kB/s 


Следующая ячейка для импорта установленных пакетов в проект

In [41]:
# use aliases for long names
import os
import requests
from github import Github as gh
from getpass import getpass
import re
import pandas as pd
import numpy as np
import base64
from io import StringIO
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler as ss
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report

## Загрузка данных 

Запросим у пользователя креды для подключения к репозиторию

In [ ]:
login = input('Enter your login: ')
password = getpass('Enter the secret value: ')

Подключимся к репозиторию

In [ ]:
repo_path = "IgorLukhnev/FarFetchRS"
# Запишем токен в переменную окружения
g_token = os.getenv('GITHUB_TOKEN', password)
# Вополним коннект
g = gh(g_token)
# Подключимся к репо
repo = g.get_repo(repo_path)

Данные будем загружать на основе датасета, подготовленного на этапе EDA

In [ ]:
dir_path = 'Data/Feature gen'
data_path = 'Data/Feature gen/data_image_embedding.csv'
contents = repo.get_contents(dir_path)
for x in contents:
    if x.path == data_path:
        data = pd.read_csv(StringIO(base64.b64decode(repo.get_git_blob(x.sha).content).decode("utf8")))

data.head()

## Подготовка данных

Для использования алгоритма кластеризации, нам необходимо сделать несколько вещей:
1. Удалить столбцы, не несущие важной информации
2. Обработать текстовые признаки (кодирование)

Начнем с шага 1.

In [ ]:
to_drop = ['Short_descr', 'img', 'cat', 'category1', 'sex', 'check']

data = data.drop(columns=to_drop)

Окей, простое сделали, теперь нужно продумать, что делаем с текстовыми значениями:
1. ffcollection -> тут кажется прикольным сделать подобие OHE с дополнительным исследованием
2. Название брэнда (по-хорошему, ради эксперимента сначала выкинем данный признак)
3. Доступные размеры - тут точно OHE

In [ ]:
# Временно уберем из данных информацию о брэнде
data_hidden = data.drop(columns=['Brand'])

Теперь посмотрим на распределение значений в столбце ffcollection

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.countplot(data=data_hidden, x='ffcollection')
plt.tight_layout()

Для unknown будет 0 во всех столбцах - для остальных 1 там, где соответсвтует

In [ ]:
for col in data_hidden['ffcollection'].value_counts().index:
    if col != 'unknown':
        data_hidden[f'col_{col}'] = (data_hidden['ffcollection'] == col).astype(int)

In [ ]:
data_hidden.head()

Аналогично поступим с размерами, возьмем следующую линейку размеров:
* XXS,
* XS,
* S,
* M,
* L,
* XL,
* XXL

In [ ]:
possible_sizes = ['XXS', 'XS', 'S', 'M', 'L', 'XL', 'XXL']

for s in possible_sizes:
    data_hidden[s] = data_hidden['avSizes'].apply(lambda x: re.search(s, x) is not None).astype(int)

In [ ]:
data_hidden.head()

Теперь полностью избавляемся от текста, нормализуем признаки

In [ ]:
data_hidden.drop(columns=['ffcollection', 'avSizes'], inplace=True)

scaler = ss()
scaled_data = scaler.fit_transform(data_hidden)

Дальше в дело вступает метод главных компонент для того, что снизить размерность.

In [ ]:
pca = PCA(n_components='mle', svd_solver='full', random_state=306)
pca.fit_transform(scaled_data)

In [ ]:
to_train = pca.transform(scaled_data)

to_train.shape

## Обучение модели

Теперь мы готовы к обучению модели

In [ ]:
scan = DBSCAN(metric='minkowski', p=3)

res1 = scan.fit_predict(to_train)

In [ ]:
km = KMeans(n_clusters=60, n_init=50, random_state=306)

res2 = km.fit_predict(to_train)

In [ ]:
# sns.countplot(res1)
sns.countplot(res2)

In [ ]:
features = pd.DataFrame(to_train)

features['target'] = res2

## Сохраняем данные

In [ ]:
features.to_csv('classified_data.csv', index=False)

## Обучим тут сразу же классификатор

In [35]:
to_drop = features['target'].value_counts()[features['target'].value_counts() <= 10].index
features.drop(index=features[features['target'].apply(lambda x: x in to_drop)].index, inplace=True)

In [44]:
clf = CatBoostClassifier(use_best_model=True, iterations=200)

X_train, X_test, y_train, y_test = train_test_split(features.drop(columns='target'), features['target'], random_state=306, test_size=0.2)

In [45]:
clf.fit(X_train, y_train, eval_set=(X_test, y_test))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.206276
0:	learn: 2.6373360	test: 2.6480743	best: 2.6480743 (0)	total: 8.96s	remaining: 29m 42s
1:	learn: 2.1785937	test: 2.1946797	best: 2.1946797 (1)	total: 13.5s	remaining: 22m 20s
2:	learn: 1.8475393	test: 1.8704847	best: 1.8704847 (2)	total: 18.1s	remaining: 19m 48s
3:	learn: 1.6189682	test: 1.6513293	best: 1.6513293 (3)	total: 22.6s	remaining: 18m 27s
4:	learn: 1.4366517	test: 1.4748098	best: 1.4748098 (4)	total: 27.2s	remaining: 17m 39s
5:	learn: 1.3045564	test: 1.3483767	best: 1.3483767 (5)	total: 31.7s	remaining: 17m 5s
6:	learn: 1.1983307	test: 1.2481502	best: 1.2481502 (6)	total: 36.2s	remaining: 16m 37s
7:	learn: 1.1196649	test: 1.1759458	best: 1.1759458 (7)	total: 40.7s	remaining: 16m 15s
8:	learn: 1.0354394	test: 1.1004652	best: 1.1004652 (8)	total: 45.1s	remaining: 15m 57s
9:	learn: 0.9673464	test: 1.0381715	best: 1.0381715 (9)	total: 49.6s	remaining: 15m 42s
10:	learn: 0.9232051	test: 1.0001172	best: 1.0001172 (10)	total: 54.1s	remaining: 15m 28s
1

In [46]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.79      0.76        14
           1       0.92      0.93      0.92        71
           2       0.99      0.99      0.99        92
           4       0.90      0.82      0.86        11
           6       0.77      0.86      0.81        51
           8       0.94      0.89      0.92        19
           9       0.91      0.90      0.91        59
          10       0.85      0.92      0.89        51
          11       0.94      0.73      0.82        44
          12       0.85      0.92      0.88        62
          13       0.91      0.86      0.88        69
          14       1.00      0.83      0.91        18
          20       0.76      0.79      0.77        28
          23       0.83      1.00      0.91         5
          24       0.86      0.91      0.89        34
          25       0.94      0.86      0.90        35
          27       0.80      0.95      0.87        21
          28       0.85    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
clf.save_model('best_classifier_ever')